In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import re
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
#

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [17]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#Convert the bonus column to numeric values, anything that isn't a number will be set to 0
df["Yearly bonus + stocks in EUR"] = pd.to_numeric(df["Yearly bonus + stocks in EUR"], errors='coerce').fillna(0)

In [19]:
#Create the `TotalComp` column which will become our target, drop the two columns we used to create the `TotalComp` column
df["TotalComp"] = df["Yearly brutto salary (without bonus and stocks) in EUR"] + df["Yearly bonus + stocks in EUR"]
df.drop(columns={"Yearly brutto salary (without bonus and stocks) in EUR", "Yearly bonus + stocks in EUR",}, inplace=True)

In [20]:
#keep the middle, over 100K and under 20K seem like outliers
df = df[(df["TotalComp"] < 1.000000e+05) & (df["TotalComp"] > 20000) ]
#remove the duplicate columns that are in the dataframe
df.drop_duplicates(inplace=True)

In [21]:
#drop Timestamp, seems like a reference column more than anything
df.drop(columns={"Timestamp"}, inplace=True)

In [22]:
#get rid of the space in `Position`
df.rename(columns ={"Position ":"Position"}, inplace=True)

In [23]:
#replace the commas that are used instead of decimals
df["Total years of experience"].replace(",",".")
df["Years of experience in Germany"].replace(",",".")
#get rid of any non numeric chars
df["Total years of experience"] = df["Total years of experience"].str.extract(r"([+-]? *(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)", expand=False)
df["Years of experience in Germany"] = df["Years of experience in Germany"].str.extract(r"([+-]? *(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)", expand=False)
df["Number of vacation days"] = df["Number of vacation days"].str.extract(r"([+-]? *(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)", expand=False)

In [24]:
# df.dtypes
# df["Number of vacation days"].value_counts(ascending=True)

In [25]:
#now make the columns numeric
df["Age"] = pd.to_numeric(df["Age"])
df["Total years of experience"] = pd.to_numeric(df["Total years of experience"])
df["Years of experience in Germany"] = pd.to_numeric(df["Years of experience in Germany"])
df["Number of vacation days"] = pd.to_numeric(df["Number of vacation days"])

In [26]:
#Stolen shamelesly from Akeem in the sample exploration file in this repo.
def replace_low_freq(d, col, threshold=10, replacement='other'):
    value_counts = d[col].value_counts() # Specific column 
    to_remove = value_counts[value_counts <= threshold].index
    tmp = d[col].replace(to_replace=to_remove, value=replacement)
    return tmp

In [27]:
#Let's create an "Other" classification for cities, Position, to help reduce the number of potential classes, speeding things up
df["City"] = replace_low_freq(df, "City", 5)
df["Position"] = replace_low_freq(df, "Position", 5)
df["Seniority level"] = replace_low_freq(df, "Seniority level", 5)
df["Your main technology / programming language"] = replace_low_freq(df, "Your main technology / programming language", 10)
df["Employment status"] = replace_low_freq(df, "Employment status", 5)
df["Main language at work"] = replace_low_freq(df, "Main language at work", 5)
df["Company type"] = replace_low_freq(df, "Company type", 10)

df["Company type"].value_counts()

Product                592
Startup                204
Consulting / Agency    112
other                   53
Name: Company type, dtype: int64

In [28]:
#Value cleanup to keep likes with likes
df["Your main technology / programming language"].replace({"Javascript": "JavaScript", "Python ":"Python"}, inplace=True)

In [64]:
#now we will make a model

y = df["TotalComp"]
x = df.drop(columns={"TotalComp"})

x_train, x_test, y_train, y_test = train_test_split(x,y)

scaler = MinMaxScaler()
encoder = OneHotEncoder()
estimator = RandomForestRegressor(n_jobs=-1)

# we'll use an imputer to get our missing numerical features filled in. The strategy is defined in the params below
numericFeaturesMeanImpute = ["Age","Total years of experience","Years of experience in Germany","Number of vacation days"]
numericMeanTransformer = Pipeline( steps=[
    ("imputeMean", SimpleImputer()),
    ("scaler", scaler)
])

categoricalFeatures = ["Gender","City","Position","Seniority level","Your main technology / programming language",
                    "Employment status", "Сontract duration", "Main language at work","Company size","Company type"]
catergoricalTransformer =  Pipeline( steps=[
    ("impute",SimpleImputer(strategy="most_frequent")),
    ("encoder", encoder)
])

columnPreProcessor = ColumnTransformer( transformers=[
    ("numMean", numericMeanTransformer, numericFeaturesMeanImpute),
    ("cat", catergoricalTransformer, categoricalFeatures)
])

# params = {  #For Extra tree regressor
#             'est__criterion':["squared_error", "friedman_mse", "absolute_error", "poisson"],
#             'est__min_samples_split':[3,4,5,6,7]
#             # 'DT__max_depth':[30,40,50,60],
#             # # 'DT__min_samples_leaf':[50,75,100,250],
#             # 'DT__criterion':["gini","entropy",'log_loss'],
#             # 'DT__ccp_alpha': [0.001,0.005,.010,.015,.020,.025,.030],
#             # # 'DT__max_samples':[.5, .6, .7]
#             # 'DT__max_leaf_nodes':[75,100,150,200]
#             }

params = {   #Random Forest
            # 'est__min_samples_split':[3,4,5,6,7],
            # # 'est__max_depth':[30,40,50,60],
            # # # 'est__min_samples_leaf':[50,75,100,250],
            # 'est__criterion':["squared_error", "friedman_mse", "absolute_error", "poisson"],
            # 'est__ccp_alpha': [0.001,0.005,.010,.015,.020,.025,.030],
            # # # 'est__max_samples':[.5, .6, .7]
            # # 'est__max_leaf_nodes':[75,100,150,200]

            'est__ccp_alpha':[0.015],
            'est__criterion':['friedman_mse'],
            'est__min_samples_split':[7],
            # 'rfe__scoring':["explained_variance","neg_mean_poisson_deviance","r2","neg_mean_absolute_error"],
            'rfe__estimator':[Lasso(), Ridge(), LinearRegression(),estimator] #Linearreg first time, ridge seccond
            }

min_features_to_select = 1
rfecv_pipe = SelectFromModel(
    estimator=estimator
)

# RFECV(
#     estimator=estimator,
#     step=1,
#     cv=3,
#     scoring="explained_variance",
#     min_features_to_select=min_features_to_select,
#     n_jobs=-1
# )


pipelineSteps = [("pre", columnPreProcessor), ('rfe', rfecv_pipe),  ('est', estimator)]
pipe = Pipeline(steps=pipelineSteps)

gridSearch = GridSearchCV(pipe, param_grid=params, cv=5, n_jobs=-1)

gridSearch.fit(x_train,y_train)
print("Train score:",gridSearch.best_score_)
predsTrain = gridSearch.predict(x_train)
mseTrain = mean_squared_error(y_train, predsTrain)
print("Train RMSE:",np.sqrt(mseTrain))
print("Test Score:", gridSearch.best_estimator_.score(x_test,y_test))
preds = gridSearch.predict(x_test)
mse = mean_squared_error(y_test, preds)
print("Test RMSE:",np.sqrt(mse))
print("Best Estimator:",gridSearch.best_estimator_)

# pipe.fit(x_train,y_train)
# print("score:",pipe.score(x_test,y_test))
# preds = pipe.predict(x_test)


In [62]:
#feature selection
# from sklearn.feature_selection import VarianceThreshold
# sel = VarianceThreshold(threshold=(.8 * (1-.8)))
# sel.fit_transform(x_train)

print(gridSearch.best_estimator_.named_steps['rfe'].n_features_in_)
print(len(gridSearch.best_estimator_.named_steps['rfe'].get_feature_names_out()))
# print(gridSearch.best_estimator_[:-1].get_feature_names_out())



#.named_steps['rfe'].


67
59


# First runs
 1. removed biggest total comp outlier, imputed nans to 0 in bonus

ExtraTrees gives me 27-42 RMSE: 11600 - 14000

Random forest gives me 43 - 57 rmse:10900 - 13000

Extra best:
`Best Estimator: Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('numMean',
                                                  Pipeline(steps=[('imputeMean',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age',
                                                   'Total years of experience',
                                                   'Years of experience in '
                                                   'Germany',
                                                   'Number of vacation days']),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  ['Gender', 'City', 'Position',
                                                   'Seniority level',
                                                   'Your main technology / '
                                                   'programming language',
                                                   'Employment status',
                                                   'Сontract duration',
                                                   'Main language at work',
                                                   'Company size',
...
                                                   'Company type'])])),
                ('est',
                 ExtraTreesRegressor(criterion='poisson', min_samples_split=4,
                                     n_jobs=-1))])`

Random Best:
`RandomForestRegressor(ccp_alpha=0.015,
                                       criterion='friedman_mse',
                                       min_samples_split=7, n_jobs=-1))])`                                

# Answers and Explainations
## Results
### Without Feature Selection
Tried two different models Random Forest and Extra Tree regression.
- Extra Tree regression: 
  - Train: R2: 37; RMSE: 2117;
  - Test: R2: 36; RMSE: 13654;
  - Time to results: 16 minutes
- Random Forest regression: 
  - Train: R2: 47; RMSE: 5962
  - Test: R2: 45; RMSE: 12229
  - Time to results: 21 minutes

### With Feature Selection
Gave up on extra tree, just did our best estimater random forest (`RandomForestRegressor(ccp_alpha=0.015,criterion='friedman_mse',min_samples_split=7, n_jobs=-1)`)
- Random forest regression RFECV:
  - Train: R2: 44; RMSE: 6225
  - Test: R2: 53; RMSE: 10939
  - Time to results: 7 minutes

---

## Feature Selection Activities
 1. Tried a `RFECV`, using the random forest. It took the features from 67 to 44.
 2. Tried a `SelectFromModel` using the random forest it tooks the features from 67 to 59
---

## Hyperparameter Changes
### Extra tree regressor (without feature selection) 
 * Tried all 4 criterion in the extra tree regressor just to see if the data is best suited for one of these over the other three
 * Tried changing the imputation strategy (mean, median, most_frequent) I was wondering if this made much of a difference, most_frequent was the eventual winner, which kind of surprised me to be honest
 * tried [3,4,5,6,7] in min_samples_split, to ensure we were combatting overfitting that can happen in trees
### Random forest (without feature changes)
 * Tried all 4 criterion in the extra tree regressor just to see if the data is best suited for one of these over the other three
### Random forest (with RFECV feature changes)
 * Tried a few different scoring types on the RFECV: 'neg_mean_squared_error',max_error,d2_pinball_score changed training R2 to 15%(!?!), by first run with `explained_variance` seems to be the winner in that group
 * Tried four different estimator models to see if any of them evaluate the feature importance better than the others. LinearRegression and Ridge traded spots in the best estimator result.
 ### Random forest (with SelectFromModel feature changes)
 * Tried four different estimator models to see if any of them evaluate the feature importance better than the others. Lasso was the repeated winner